# TensorRT Quantization Tutorial

This notebook is designed to show the features of the TensorRT passes integrated into MASE as part of the MASERT framework.

## Section 1. INT8 Quantization
Firstly, we will show you how to do a int8 quantization of a simple model, `jsc-toy`, and compare the quantized model to the original model using the `Machop API`. The quantization process is split into the following stages, each using their own individual pass, and are explained in depth at each subsection:

1. [Fake quantization](#section-11-fake-quantization): `tensorrt_fake_quantize_transform_pass`
2. [Calibration](#section-12-calibration): `tensorrt_calibrate_transform_pass`
3. [Quantized Aware Training](#section-13-quantized-aware-training-qat): `tensorrt_fine_tune_transform_pass`
4. [Quantization](#section-14-tensorrt-quantization): `tensorrt_engine_interface_pass`
5. [Analysis](#section-15-performance-analysis): `tensorrt_analysis_pass`

We start by loading in the required libraries and passes required for the notebook as well as ensuring the correct path is set for machop to be used.

In [1]:
import sys
import os
from pathlib import Path
import toml

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    runtime_analysis_pass,
    )

set_logging_verbosity("info")

[2024-03-28 07:10:18,870] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0328 07:10:21.221665 140447335733056 logger.py:44] Set logging level to info


Next, we load in the toml file used for quantization. To view the configuration, click [here](../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml).

In [2]:
# Path to your TOML file
JSC_TOML_PATH = '../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml'

# Reading TOML file and converting it into a Python dictionary
with open(JSC_TOML_PATH, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt' section and its children
tensorrt_config = pass_args.get('passes', {}).get('tensorrt', {})
# Extract the 'passes.runtime_analysis' section and its children
runtime_analysis_config = pass_args.get('passes', {}).get('runtime_analysis', {})

We then create a `MaseGraph` by loading in a model and training it using the toml configuration model arguments.

In [3]:
# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_config, runtime_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['dataset'] = pass_args['dataset']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)


input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Next, we train the `jsc-toy` model using the machop `train` action with the config from the toml file.

In [4]:
# !ch train --config {JSC_TOML_PATH}

Then we load in the checkpoint. You will have to adjust this according to where it has been stored in the mase_output directory.

In [5]:
# Load in the trained checkpoint - change this accordingly
JSC_CHECKPOINT_PATH = "../../../mase_output/jsc-toy_cls_jsc-pre-trained/best.ckpt"

model = load_model(load_name=JSC_CHECKPOINT_PATH, load_type="pl", model=model)

# Initiate metadata
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)
mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})

# Before we begin, we will copy the original MaseGraph model to use for comparison during quantization analysis
mg_original = deepcopy_mase_graph(mg)

INFO     Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy_cls_jsc-pre-trained/best.ckpt
I0328 07:10:23.969384 140447335733056 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy_cls_jsc-pre-trained/best.ckpt


### Section 1.1 Fake Quantization

Firstly, we fake quantize the module in order to perform calibration and fine tuning before actually quantizing - this is only used if we have int8 calibration as other precisions are not currently supported within [pytorch-quantization](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#) library.

This is acheived through the `tensorrt_fake_quantize_transform_pass` which goes through the model, either by type or by name, replaces each layer appropriately to a fake quantized form if the `quantize` parameter is set in the default config (`passes.tensorrt_quantize.default.config`) or on a per name or type basis. 

Currently the quantizable layers are:
- Linear
- Conv1d, Conv2d, Conv3d 
- ConvTranspose1d, ConvTranspose2d, ConvTranspose3d 
- MaxPool1d, MaxPool2d, MaxPool3d
- AvgPool1d, AvgPool2d, AvgPool3d
- LSTM, LSTMCell

To create a custom quantized module, click [here](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#document-tutorials/creating_custom_quantized_modules).


In [6]:
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_config)
summarize_quantization_analysis_pass(mg_original, mg)

INFO     Applying fake quantization to PyTorch model...
I0328 07:10:24.099733 140447335733056 utils.py:240] Applying fake quantization to PyTorch model...
INFO     Fake quantization applied to PyTorch model.
I0328 07:10:24.360137 140447335733056 utils.py:265] Fake quantization applied to PyTorch model.
INFO     Quantized graph histogram:
I0328 07:10:24.372440 140447335733056 summary.py:84] Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       4 |         0 |           4 |
| Linear          | linear       |       3 |         3 |           0 |
| ReLU            | relu         |       4 |         0 |           4 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |
I0328 07:10:24.374239 140447335733056 summary.py:85] 
| Original type   |

As you can see we have succesfully fake quantized all linear layers inside `jsc-toy`. This means that we will be able to simulate a quantized model in order to calibrate and fine tune it. This fake quantization was done on typewise i.e. for linear layers only. See [Section 4](#section-4-layer-wise-mixed-precision) for how to apply quantization layerwise - i.e. only first and second layers for example.

### Section 1.2 Calibration

Next, we perform calibration using the `tensorrt_calibrate_transform_pass`. Calibration is achieved by passing data samples to the quantizer and deciding the best amax for activations. 

Calibrators can be added as a search space parameter to examine the best performing calibrator. The calibrators have been included in the toml as follows.
For example: `calibrators = ["percentile", "mse", "entropy"]`

Note: 
- To use `percentile` calibration, a list of percentiles must be given
- To use `max` calibration, the `histogram` weight and input calibrators must be removed and replaced with `max`. This will use global maximum absolute value to calibrate the model.
- If `post_calibration_analysis` is set true the `tensorrt_analysis_pass` will be run for each calibrator tested to evaluate the most suitable calibrator for the model.

In [7]:
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_config)

INFO     Starting calibration of the model in PyTorch...
I0328 07:10:24.383733 140447335733056 calibrate.py:142] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0328 07:10:24.387609 140447335733056 calibrate.py:151] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0328 07:10:24.388971 140447335733056 calibrate.py:151] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0328 07:10:24.390182 140447335733056 calibrate.py:151] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0328 07:10:24.391498 140447335733056 calibrate.py:151] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0328 07:10:24.392848 140447335733056 calibrate.py:151] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization an

From the results, the 99% `percentile` clips too many values during the amax calibration, compromising the loss. However 99.99% demonstrates higher validation accuracy alongside `mse` and `entropy` for `jsc-toy`. For such a small model, the methods are not highly distinguished, however for larger models this calibration process will be important for ensuring the quantized model still performs well. 

### Section 1.3 Quantized Aware Training (QAT)

The `tensorrt_fine_tune_transform_pass` is used to fine tune the quantized model. 

For QAT it is typical to employ 10% of the original training epochs, starting at 1% of the initial training learning rate, and a cosine annealing learning rate schedule that follows the decreasing half of a cosine period, down to 1% of the initial fine tuning learning rate (0.01% of the initial training learning rate). However this default can be overidden by setting the `epochs`, `initial_learning_rate` and `final_learning_rate` in `passes.tensorrt_quantize.fine_tune`.

The fine tuned checkpoints are stored in the ckpts/fine_tuning folder:

```
mase_output
└── tensorrt
    └── quantization
        └──model_task_dataset_date
            ├── cache
            ├── ckpts
            │   └── fine_tuning
            ├── json
            ├── onnx
            └── trt
```

In [8]:
# mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_config)

### Section 1.4 TensorRT Quantization

After QAT, we are now ready to convert the model to a tensorRT engine so that it can be run with the superior inference speeds. To do so, we use the `tensorrt_engine_interface_pass` which converts the `MaseGraph`'s model from a Pytorch one to an ONNX format as an intermediate stage of the conversion.

During the conversion process, the `.onnx` and `.trt` files are stored to their respective folders shown in [Section 1.3](#section-13-quantized-aware-training-qat).

This interface pass returns a dictionary containing the `onnx_path` and `trt_engine_path`.

In [9]:
mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_config)

INFO     Converting PyTorch model to ONNX...
I0328 07:10:58.712838 140447335733056 quantize.py:159] Converting PyTorch model to ONNX...
/root/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:363: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if min_amax < 0:
/root/anaconda3/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:366: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax

### Section 1.5 Performance Analysis

To showcase the improved inference speeds and to evaluate accuracy and other performance metrics, the `tensorrt_analysis_pass` can be used.

The tensorRT engine path obtained the previous interface pass is now inputted into the the analysis pass. The same pass can take a MaseGraph as an input, as well as an ONNX graph. For this comparison, we will first run the anaylsis pass on the original unquantized model and then on the int8 quantized model.

In [10]:
_, _ = runtime_analysis_pass(mg_original, pass_args=runtime_analysis_config)
_, _ = runtime_analysis_pass(meta['trt_engine_path'], pass_args=runtime_analysis_config)

INFO     Starting transformation analysis on jsc-toy
I0328 07:11:53.804656 140447335733056 runtime_analysis.py:309] Starting transformation analysis on jsc-toy
INFO     
Results jsc-toy:
+------------------------------+--------------+
|      Metric (Per Batch)      |    Value     |
+------------------------------+--------------+
|    Average Test Accuracy     |   0.71971    |
|      Average Precision       |   0.71884    |
|        Average Recall        |   0.71127    |
|       Average F1 Score       |   0.71274    |
|         Average Loss         |    0.8116    |
|       Average Latency        |  1.5855 ms   |
|   Average GPU Power Usage    |   22.883 W   |
| Inference Energy Consumption | 0.010078 mWh |
+------------------------------+--------------+
I0328 07:11:59.751923 140447335733056 runtime_analysis.py:437] 
Results jsc-toy:
+------------------------------+--------------+
|      Metric (Per Batch)      |    Value     |
+------------------------------+--------------+
|    Average

As shown above, the latency has decreased around 6x with the `jsc-toy` model without compromising accuracy due to the well calibrated amax and quantization-aware fine tuning and additional runtime optimizations from TensorRT. The inference energy consumption has thus also dropped tremendously and this is an excellent demonstration for the need to quantize in industry especially for LLMs in order to reduce energy usage. 

## Section 2. FP16 Quantization

We will now load in a new toml configuration that uses fp16 instead of int8, whilst keeping the other settings the exact same for a fair comparison. This time however, we will use chop from the terminal which runs all the passes showcased in [Section 1](#section-1---int8-quantization).

Since float quantization does not require calibration, nor is it supported by `pytorch-quantization`, the model will not undergo fake quantization; for the time being this unfortunately means QAT is unavailable and only undergoes Post Training Quantization (PTQ). 

In [14]:
JSC_FP16_BY_TYPE_TOML = "../../../machop/configs/tensorrt/jsc_toy_FP16_quantization_by_type.toml"
!ch transform --config {JSC_FP16_BY_TYPE_TOML} --load {JSC_CHECKPOINT_PATH} --load-type pl

8808.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
[2024-03-28 09:37:03,989] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0328 09:37:06.938809 140201001654080 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------------------+--------------------------+--------------------------+
| Name                    |        Default         |       Config. File       |     Manual Override      |        Effective         |
+-------------------------+------------------------+--------------------------+--------------------------+--------------------------+
| task                    |     classification     |           cls            |                          |           cls            |
| load_name               |          None          | ../mase_output/jsc-toy_c | /root/mase/mase_output/j | /root/mase/mase_output/j |
|                         |    

As you can see, `fp16` acheives a slighty higher test accuracy but a slightly lower latency (~30%) from that of int8 quantization; it is still ~2.5x faster than the unquantized model. Now lets apply quantization to a more complicated model.

## Section 3. Type-wise Mixed Precision on Larger Model
We will now quantize `vgg7` which includes both convolutional and linear layers, however for this demonstration we want to quantize all layer types except the linear layers.

In this case, we set:

- The `by` parameter to `type`
- The `quantize` parameter to true for `passes.tensorrt_quantize.conv2d.config` and `precision` parameter to 'int8'.
- The `input` and `weight` quantize axis for the conv2d layers.
- The default `passes.tensorrt_quantize.default.config` precision to true. 

During the TensorRT quantization, the model's conv2d layers will be converted to an int8 fake quantized form, whilst the linear layers are kept to their default 'fp16'. Calibration of the conv2d layers will be undergone and fine tuning.  

You may either download a pretrained model [here](https://imperiallondon-my.sharepoint.com/:f:/g/personal/zz7522_ic_ac_uk/Emh3VT7Q_qRFmnp8kDrcgDoBwGUuzLwwKNtX8ZAt368jJQ?e=gsKONa), otherwise train it yourself as shown below. 

In [ ]:
!ch train --config ../../../machop/configs/tensorrt/vgg7_layerwise_mixed_precision.toml.toml

We will now load the checkpoint in, quantize the model and compare it to the unquantized version as we did in [Section 1.5](#section-15-performance-analysis)

In [3]:
VGG_CHECKPOINT_PATH = "../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt"

In [7]:
!ch transform --config ../../../machop/configs/tensorrt/vgg7_typewise_mixed_precision.toml --load {VGG_CHECKPOINT_PATH} --load-type pl

[2024-03-18 15:52:43,166] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0318 15:52:44.743802 139761495775040 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| Name                    |        Default         | Config. File |     Manual Override      |        Effective         |
+-------------------------+------------------------+--------------+--------------------------+--------------------------+
| task                    |     classification     |     cls      |                          |           cls            |
| load_name               |          None          |              | /root/mase/mase_output/v | /root/mase/mase_output/v |
|                         |                        |              |  gg7-pre-trained/test-   |  gg7-pre-trained/test-   |
|                         |                        |            

## Section 4. Layer-wise Mixed Precision

So far we have strictly quantized either in int8 or fp16. Now, we will show how to conduct layerwise mixed precision using the same `vgg7` model. In this case we will show how for instance, layer 0 and 1 can be set to fp16, while layer 2 and 3 can be int8 quantized. 

For this, we set:
- The `by` parameter to `name`
- The `precision` to 'fp16' for `passes.tensorrt_quantize.feature_layers_0.config and passes.tensorrt_quantize.feature_layers_1.config`
- The `precision` to 'int8' for `passes.tensorrt_quantize.feature_layers_0.config and passes.tensorrt_quantize.feature_layers_1.config`



In [1]:
import sys
import os
from pathlib import Path
import toml
from copy import copy, deepcopy

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    runtime_analysis_pass,
    )

set_logging_verbosity("info")

/root/anaconda3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-19 16:33:49,791] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0319 16:33:51.317272 140297220093760 logger.py:44] Set logging level to info


In [2]:
# Path to your TOML file
# toml_file_path = '../../../machop/configs/tensorrt/vgg7_layerwise_mixed_precision.toml'
toml_file_path = '../../../machop/configs/tensorrt/vgg7_typewise_mixed_precision.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt' section and its children
tensorrt_config = pass_args.get('passes', {}).get('tensorrt', {})
# Extract the 'passes.runtime_analysis' section and its children
runtime_analysis_config = pass_args.get('passes', {}).get('runtime_analysis', {})

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_config, runtime_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Load in the trained checkpoint - change this accordingly
VGG_CHECKPOINT_PATH = "../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt"

model = load_model(load_name=VGG_CHECKPOINT_PATH, load_type="pl", model=model)

# Initiate metadata
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)
mg, _ = init_metadata_analysis_pass(mg, None)

mg_original = deepcopy_mase_graph(mg)

mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)
mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})

INFO     Loaded pytorch lightning checkpoint from ../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt
I0318 15:33:56.843245 140155325249344 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from ../../../mase_output/vgg7-pre-trained/test-accu-0.9332.ckpt


In [10]:
_, _ = runtime_analysis_pass(mg, pass_args=runtime_analysis_config)

INFO     Starting transformation analysis
I0318 09:55:37.107798 140195009255232 analysis.py:214] Starting transformation analysis
INFO     
Results vgg7:
+------------------------------+-------------+
|            Metric            |    Value    |
+------------------------------+-------------+
|    Average Test Accuracy     |   0.91967   |
|      Average Precision       |   0.92315   |
|        Average Recall        |   0.92362   |
|       Average F1 Score       |   0.92326   |
|         Average Loss         |   0.23674   |
|       Average Latency        |  28.123 ms  |
|   Average GPU Power Usage    |  100.17 W   |
| Inference Energy Consumption | 0.78256 mWh |
+------------------------------+-------------+
I0318 09:55:40.969246 140195009255232 analysis.py:330] 
Results vgg7:
+------------------------------+-------------+
|            Metric            |    Value    |
+------------------------------+-------------+
|    Average Test Accuracy     |   0.91967   |
|      Average Precision

In [4]:
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_config)
# summarize_quantization_analysis_pass(mg_original, mg)
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_config)
mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_config)
mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_config)

INFO     Applying fake quantization to PyTorch model...
I0318 15:34:00.409916 140155325249344 utils.py:168] Applying fake quantization to PyTorch model...
INFO     Fake quantization applied to PyTorch model.
I0318 15:34:00.682099 140155325249344 utils.py:193] Fake quantization applied to PyTorch model.
INFO     Starting calibration of the model in PyTorch...
I0318 15:34:00.683382 140155325249344 calibrate.py:91] Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
I0318 15:34:00.694193 140155325249344 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 15:34:00.695298 140155325249344 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
I0318 15:34:00.695932 140155325249344 calibrate.py:100] Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibrat

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


I0318 15:34:13.796578 140155325249344 cuda.py:61] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I0318 15:34:13.808069 140155325249344 model_summary.py:94] 
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | GraphModule        | 14.0 M
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | acc_train | MulticlassAccuracy | 0     
3 | loss_val  | MeanMetric         | 0     
4 | loss_test | MeanMetric         | 0     
-------------------------------------------------
14.0 M    Trainable params
0         Non-trainable params
14.0 M    Total params
56.118    Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 13.05it/s]

/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


...

Epoch 1: 100%|██████████| 196/196 [00:43<00:00,  4.52it/s, v_num=8, train_acc_step=0.863, val_acc_epoch=0.893, val_loss_epoch=0.223]

INFO     Fine Tuning Complete
I0318 15:35:42.213792 140155325249344 fine_tune.py:120] Fine Tuning Complete
INFO     Converting PyTorch model to ONNX...
I0318 15:35:42.219084 140155325249344 quantize.py:129] Converting PyTorch model to ONNX...


/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:363: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if min_amax < 0:
/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:366: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
/opt/conda/envs/mase/lib/python3.11/site-packages/pytorch_quantization/tensor_quant.py:376: TracerWarning: Converting a tensor to a Pytho

[03/18/2024-15:35:48] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.


INFO     TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/tensorrt/quantization/trt/2024_03_18/version_10/model.trt
I0318 15:38:52.261729 140155325249344 quantize.py:124] TensorRT Conversion Complete. Stored trt model to /root/mase/mase_output/tensorrt/quantization/trt/2024_03_18/version_10/model.trt
INFO     TensorRT Model Summary Exported to /root/mase/mase_output/tensorrt/quantization/json/2024_03_18/version_10/model.json
I0318 15:38:52.553308 140155325249344 quantize.py:168] TensorRT Model Summary Exported to /root/mase/mase_output/tensorrt/quantization/json/2024_03_18/version_10/model.json


In [5]:
_, _ = runtime_analysis_pass(meta['trt_engine_path'], pass_args=runtime_analysis_config)

INFO     
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 3, 32, 32)       | (256, 3, 32, 32)       | input
1     | Output  | FLOAT    | (256, 10)              | (256, 10)              | 220
I0318 15:38:52.609910 140155325249344 analysis.py:117] 
TensorRT Engine Input/Output Information:
Index | Type    | DataType | Static Shape         | Dynamic Shape        | Name
------|---------|----------|----------------------|----------------------|-----------------------
0     | Input   | FLOAT    | (256, 3, 32, 32)       | (256, 3, 32, 32)       | input
1     | Output  | FLOAT    | (256, 10)              | (256, 10)              | 220
INFO     Starting transformation analysis
I0318 15:38:52.611444 140155325249344 analysis.py:214] Starting transformation analysis


INFO     
Results vgg7-quantized:
+------------------------------+-----------+
|            Metric            |   Value   |
+------------------------------+-----------+
|    Average Test Accuracy     |  0.93222  |
|      Average Precision       |  0.93206  |
|        Average Recall        |  0.93222  |
|       Average F1 Score       |  0.93204  |
|         Average Loss         |  0.22506  |
|       Average Latency        | 19.479 ms |
|   Average GPU Power Usage    | 110.15 W  |
| Inference Energy Consumption | 0.596 mWh |
+------------------------------+-----------+
I0318 15:38:55.434816 140155325249344 analysis.py:323] 
Results vgg7-quantized:
+------------------------------+-----------+
|            Metric            |   Value   |
+------------------------------+-----------+
|    Average Test Accuracy     |  0.93222  |
|      Average Precision       |  0.93206  |
|        Average Recall        |  0.93222  |
|       Average F1 Score       |  0.93204  |
|         Average Loss         

## Section 5. Language Models

In [2]:
# Path to your TOML file
toml_file_path = '../../../machop/configs/tensorrt/opt-125M_layerwise_mixed_precision_by_name.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt' section and its children
tensorrt_config = pass_args.get('passes', {}).get('tensorrt', {})
# Extract the 'passes.runtime_analysis' section and its children
runtime_analysis_config = pass_args.get('passes', {}).get('runtime_analysis', {})

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

opt_tokenizer = get_tokenizer("facebook/opt-125m")

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0, # os.cpu_count()
    max_token_len=128,
    tokenizer=opt_tokenizer,
    load_from_cache_file=True,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_config, runtime_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

In [3]:
model_info = get_model_info(model_name)
model = get_model(
    "facebook/opt-125m:patched",
    task="lm",
    dataset_info=get_dataset_info("wikitext2"),
    pretrained=True,
)

# Load in the trained checkpoint - change this accordingly
# OPT125M_CHECKPOINT_PATH = "../../../mase_output/jsc-toy_classification_jsc_2024-03-17/software/training_ckpts/best.ckpt"
# model = load_model(load_name=OPT125M_CHECKPOINT_PATH, load_type="pl", model=model)

model_info = get_model_info("facebook/opt-125m:patched")
cf_args = get_cf_args(model_info=model_info, task="lm", model=model)

mg = MaseGraph(model=model, cf_args=cf_args)

# dummy_in = get_dummy_input(model_info, data_module=data_module, task="lm")
# if len(mg.model.additional_inputs) > 0:
#     dummy_in = dummy_in | mg.model.additional_inputs

# Initiate metadata
mg, _ = init_metadata_analysis_pass(mg, pass_args=None)

# # Before we begin, we will copy the original MaseGraph model to use for comparison during quantization analysis
# mg_original = deepcopy_mase_graph(mg)

In [ ]:
# _, _ = runtime_analysis_pass(mg, pass_args=runtime_analysis_config)

In [4]:
# mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_config)
# summarize_quantization_analysis_pass(mg_original, mg)

# mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_config)

# mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_config)

mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_config)

_, _ = runtime_analysis_pass(mg_original, pass_args=runtime_analysis_config)
_, _ = runtime_analysis_pass(meta['trt_engine_path'], pass_args=runtime_analysis_config)

INFO     Converting PyTorch model to ONNX...
I0318 08:54:47.010779 139760749061952 quantize.py:129] Converting PyTorch model to ONNX...
ERROR:tornado.general:SEND Error: Host unreachable
Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2070, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2106, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt
